<a href="https://colab.research.google.com/github/ratabrak/siriusAI/blob/main/SiriusAI_dopWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets

In [ ]:
import torch
import os
import requests
from PIL import Image
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel

In [ ]:
f = os.makedirs('data', exist_ok=True)
model = CLIPModel.from_pretrained("Marqo/marqo-fashionCLIP")
processor = CLIPProcessor.from_pretrained("Marqo/marqo-fashionCLIP")
ds = load_dataset("ceyda/fashion-products-small")
for example in ds['train']:
  with open(f"data/{example['filename']}", 'wb') as smt:
    example['image'].save(smt, format='jpeg')

In [ ]:
def describe_clothing(image_path):
    # Открываем изображение
    image = Image.open(image_path).convert("RGB")

    # Пример списка текстовых описаний одежды
    descriptions = [
        "A red dress with floral patterns",
        "A casual blue denim jacket",
        "A black leather jacket",
        "A white cotton T-shirt",
        "A pair of blue jeans",
        "A formal black suit",
        "A yellow summer dress",
    ]

    # Применяем процессор для изображения и текста
    inputs = processor(
        text=descriptions,
        images=image,
        return_tensors="pt",
        padding=True
    )

    # Прогоняем через модель
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # Логиты для изображения
    probs = logits_per_image.softmax(dim=1)  # Конвертируем в вероятности

    # Находим наиболее вероятное описание
    best_description_index = probs.argmax().item()
    predicted_description = descriptions[best_description_index]

    return f"Описание одежды: {predicted_description}"

dirname = 'data2'
for fname in os.listdir(dirname):
  print(describe_clothing(dirname + '/' + fname))